# Etapa da Ingestão
Etapa da ingestão dos dados.

Extração dos dados da [API da Marvel](https://developer.marvel.com/), transformação inicial desses dados e carregamento no disco.

<img align='center' src='../figures/marvel.png' style='width:800px;'>

> **Nota**: **Artigo no Medium** da etapa da `ingestão` desse sistema em português: [Análise de Sentimentos Sobre os Quadrinhos da Marvel (Parte 1) - Ingestão, EDA e Pré-processamento](https://medium.com/@guineves.py/c5a0e35bb586).

## Table of Contents
* [Pacotes](#1)
* [Variáveis de Ambiente](#2)
* [Extração, Transformação e Carregamento](#3)
    * [Extração dos Dados](#3.1)
    * [Transformação dos Dados](#3.2)
        * [Rotulação de Dados](#3.2.1)
            * [Zero-Shot Learning](#3.2.1.1)
    * [Carregamento dos Dados](#3.3)

<a name="1"></a>
## Pacotes
Pacotes que foram utilizados no sistema:
* [os](https://docs.python.org/3/library/os.html): módulo integrado, fornece uma maneira portátil de usar funcionalidades dependentes do sistema operacional;
* [sys](https://docs.python.org/3/library/sys.html): fornece acesso a algumas variáveis usadas ou mantidas pelo interpretador e a funções que interagem fortemente com o interpretador;
* [requests](https://pypi.org/project/requests/): é uma biblioteca HTTP simples, para fazer solicitações HTTP;
* [haslib](https://docs.python.org/3/library/hashlib.html): implementa uma interface comum para muitos algoritmos diferentes de hash seguro e resumo de mensagens;
* [time](https://docs.python.org/3/library/time.html): fornece várias funções relacionadas ao tempo;
* [dotenv](https://pypi.org/project/python-dotenv/): lê pares de chave-valor de um arquivo .env e pode defini-los como variáveis de ambiente;
* [pandas](https://pandas.pydata.org/): é o principal pacote para manipulação de dados;
* [numpy](www.numpy.org): é o principal pacote para computação científica;
* [transformers](https://huggingface.co/docs/transformers/index): fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração;
* [src](../src/): pacote com todos os códigos de todas as funções utilitárias criadas para esse sistema. Localizado dentro do diretório `../src/`.

In [66]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import numpy as np
from transformers import pipeline

PROJECT_ROOT = os.path.abspath( # Obtendo a versão absoluta normalizada do path raíz do projeto
    os.path.join( # Concatenando os paths
        os.getcwd(), # Obtendo o path do diretório dos notebooks
        os.pardir # Obtendo a string constante usada pelo OS para fazer referência ao diretório pai
    )
)
# Adicionando o path à lista de strings que especifica o path de pesquisa para os módulos
sys.path.append(PROJECT_ROOT)
from src.ingestion import *

> **Nota**: os códigos para as funções utilitárias utilizadas nesse sistema estão no script `ingestion.py` dentro do diretório `../src/`.

<a name="3"></a>
## Variáveis de Ambiente
Definindo as variáveis de ambiente:
* `MARVEL_PUBLIC_KEY`: a public key para conexão e uso das APIs.
* `MARVEL_PRIVATE_KEY`: a private key para conexão e uso das APIs.

In [16]:
PUBLIC_KEY = str(os.environ['MARVEL_PUBLIC_KEY'])
PRIVATE_KEY = str(os.environ['MARVEL_PRIVATE_KEY'])

<a name="3"></a>
## Extração, Transformação e Carregamento
ETL (Extração, Transformação e Carregamento), requisitando a autorização da Marvel, extraindo os dados dos personagens e dos comics da Marvel, transformando esses dados e carregamando os dados no disco.

<a name="3.1"></a>
### Extração dos Dados
Inicializando a classe `MarvelIngestion`, que é composta pela função `get_params`, que define as variáveis, o hash e os parâmetros para a requisição à API, o método `__call__` que realiza a conexão e a chamada à API para extrair os dados solicitados.

In [234]:
ingestion = MarvelIngestion(PUBLIC_KEY, PRIVATE_KEY)

Extraindo os dados dos comics e, suas respectivas descrições e, em seguida, projetando os 5 primeiros exemplos do dataset dos comics `df_comics`.

In [132]:
df_comics = ingestion(endpoint='comics', format_='comic')
print(f'Dimensão do comics dataset: {df_comics.shape}')
df_comics.head()

Dimensão do comics dataset: (18802, 3)


,id,title,description
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
1,94801,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
2,94802,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
3,94803,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
4,94804,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...


<a name="3.2"></a>
### Transformação dos Dados
Como podemos ver acima, as features `title` e `description` do dataset dos comics tem dados duplicados, independente se o seu `id` é diferente. Portanto, vamos excluir esses exemplos duplicados.

Primeiro, vamos transformar a feature `id` no índice desse dataset para contar os exemplos duplicados entre as features `title` e `description`.

In [90]:
df_comics = df_comics.set_index('id')
df_comics.head()

,title,description
id,,
94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
94801,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
94802,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
94803,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
94804,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...


Contando os exemplos duplicados.

In [46]:
print(f'Número de exemplos duplicados: {df_comics.duplicated().sum()}')

Número de exemplos duplicados: 1069


Deletando os exemplos duplicados.

In [78]:
df_comics = df_comics.drop_duplicates()
print(f'Número de exemplos duplicados: {df_comics.duplicated().sum()}')
df_comics = df_comics.reset_index().drop(columns=['index'])
df_comics.head()

Número de exemplos duplicados: 0


,id,title,description
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
1,93339,The Mighty Valkyries (2021) #3,CHILDREN OF THE AFTERLIFE! While Kraven the Hu...
2,94884,The Mighty Valkyries (2021) #3 (Variant),CHILDREN OF THE AFTERLIFE! While Kraven the Hu...
3,93350,X-Corp (2021) #2,A SHARK IN THE WATER! After X-CORP’s shocking ...
4,94896,X-Corp (2021) #2 (Variant),A SHARK IN THE WATER! After X-CORP?s shocking ...


Checando as duplicatas apenas para a feature `description` de cada comic.

In [42]:
print(f'Número de exemplos duplicados: {df_comics[["description"]].duplicated().sum()}\nDimensão do dataset: {df_comics.shape}')

Número de exemplos duplicados: 806
Dimensão do dataset: (17733, 3)


Deletando os exemplos duplicados.

In [82]:
df_comics = df_comics.drop_duplicates('description')
print(f'Dimensão do dataset após a exclusão dos exemplos duplicados: {df_comics.shape}')

Dimensão do dataset após a exclusão dos exemplos duplicados: (16927, 3)


Checando as duplicatas apenas para a `description` de cada comic após excluir os exemplos duplicados.

In [80]:
print(f'Número de exemplos duplicados: {df_comics[["description"]].duplicated().sum()}')
df_comics.head()

Número de exemplos duplicados: 0


,id,title,description
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...
1,93339,The Mighty Valkyries (2021) #3,CHILDREN OF THE AFTERLIFE! While Kraven the Hu...
2,94884,The Mighty Valkyries (2021) #3 (Variant),CHILDREN OF THE AFTERLIFE! While Kraven the Hu...
3,93350,X-Corp (2021) #2,A SHARK IN THE WATER! After X-CORP’s shocking ...
4,94896,X-Corp (2021) #2 (Variant),A SHARK IN THE WATER! After X-CORP?s shocking ...


Estamos com 0 duplicatas, mas podemos ver alguns exemplos duplicados na feature `description`, vamos analisar isso com mais detalhes.

In [39]:
print(f'Algum exemplo:\n{df_comics["description"][1]}\n')
print(f'Exemplo duplicado:\n{df_comics["description"][2]}')

Algum exemplo:
CHILDREN OF THE AFTERLIFE! While Kraven the Hunter stalks Jane Foster on Midgard and the newest Valkyrie fights for her soul on Perdita, Karnilla, the queen of Hel, works a miracle in the land of the dead! But Karnilla isn’t Hel’s only ruler—and now she’s upset the cosmic balance. There will be a price to pay…and Karnilla intends to ensure the Valkyries pay it.

Exemplo duplicado:
CHILDREN OF THE AFTERLIFE! While Kraven the Hunter stalks Jane Foster on Midgard and the newest Valkyrie fights for her soul on Perdita, Karnilla, the queen of Hel, works a miracle in the land of the dead! But Karnilla isn?t Hel?s only ruler?and now she?s upset the cosmic balance. There will be a price to pay?and Karnilla intends to ensure the Valkyries pay it.


O que diferencia os exemplos duplicados são algumas pontuações. Portanto, não vamos tratar esse problema agora. Esse problema será tratado posteriormente no pré-processamento desses dados.

<a name="3.2.1"></a>
#### Rotulação de Dados
Precisamos rotular os dados extraídos e sem duplicatas. Portanto, podemos rotular exemplo por exemplo manualmente (chamamos esse método de `in-house`), ou encontramos uma empresa para rotular esses dados (chamamos esse método de `outsourced`), ou usamos uma plataforma crowdsourcing, para que um grande grupo rotule coletivamente os dados (chamamos esse método de `crowdsourced`), ou usamos o método de `zero-shot learning`, que foi o método utilizado nesse sistema.

Definindo os labels e selecionando os textos do dataset.

In [37]:
labels = [
    'action',
    'non-action',
]

corpus_comics = df_comics['description'].tolist()
print(f'Tamanho do corpus dos comics: {len(corpus_comics)}')

Tamanho do corpus dos comics: 16927


<a name="3.2.1.1"></a>
##### Zero-Shot Learning
Tradicionalmente, o `zero-shot learning (ZSL)` geralmente se referia a um tipo bastante específico de tarefa, aprender um classificador em um set de labels e depois avaliar em um set diferente de labels que o classificador nunca viu antes. Recentemente, especialmente em NLP, tem sido usado de forma muito mais ampla, fazendo com que um modelo faça algo para o qual não foi explicitamente treinado. Um exemplo bem conhecido disso está no paper do GPT-2 (Generative Pretrained Transformer-2, OpenAI), onde os autores avaliam um modelo de linguagem em tarefas posteriores, como neural machine translation (NMT), sem fine-tuning direto nessas tarefas.

Digamos que temos um modelo de embedding, um set de possíveis classes e a métrica de similaridade de cosseno. Então, classificamos uma determinada sequência de acordo com:
$$\hat{c} = \mathrm{arg} \max_{c\ \epsilon\ \mathrm{C}} \cos( \phi_{\mathrm{enviado}}(\vec{x}) Z,\ \phi_{\mathrm{enviado}}(c) Z)$$
* $\phi_{\mathrm{enviado}}$: modelo de embedding.
* $\mathrm{C}$: set com possíveis classes.
* $\mathrm{cos}$: métrica de similaridade de cosseno.
* $\vec{x}$: determinada sequência.
> **Referência**: [Zero-Shot Learning in Modern NLP](https://joeddav.github.io/blog/2020/05/29/ZSL.html)

Portanto, usamos o método `zero-shot learning` para rotular os nossos dados.

Definindo a pipeline com a tarefa de `zero-shot-classification` e o modelo `facebook/bart-large-mnli`.

In [66]:
pipe_bart = pipeline(
    'zero-shot-classification',
    model='facebook/bart-large-mnli'
)

Device set to use cpu


Executando a tarefa de zero-shot learning para rotular os dados do dataset.

In [35]:
output_bart_comics = pipe_bart(corpus_comics, labels)
print(f'Tamanho do vetor de labels do corpus dos comics: {len(output_bart_comics)}')

Tamanho do vetor de labels do corpus dos comics: 16927


Percorrendo o output do modelo, selecionando o id do label com o maior score de cada exemplo, selecionando o id na lista de labels, adicionando o label em uma lista para adicionar ao dataset final e plotando os primeiros 5 exemplos do dataset.

In [31]:
# Lista para armazenar o label de cada exemplo
labels_comic = []

# Percorrendo o output do modelo
for i in range(len(output_bart_comics)):
    idx = np.argmax(output_bart_comics[i]['scores']) # Selecionando o id do label com o maior score de cada exemplo
    label = output_bart_comics[i]['labels'][idx] # Selecionando o id na lista de labels
    labels_comic.append(label) # Adicionando em uma lista para adicionar ao dataset final

# Adicionando os labels ao dataset final
df_comics['y'] = labels_comic
df_comics.head()

,id,title,description,y
0,94799,Demon Days: Mariko (2021) #1 (Variant),IN THE SHADOW OF KIRISAKI MOUNTAIN?A SECRET HI...,non-action
1,93339,The Mighty Valkyries (2021) #3,CHILDREN OF THE AFTERLIFE! While Kraven the Hu...,action
2,94884,The Mighty Valkyries (2021) #3 (Variant),CHILDREN OF THE AFTERLIFE! While Kraven the Hu...,action
3,93350,X-Corp (2021) #2,A SHARK IN THE WATER! After X-CORP’s shocking ...,non-action
4,94896,X-Corp (2021) #2 (Variant),A SHARK IN THE WATER! After X-CORP?s shocking ...,non-action


<a name="3.3"></a>
### Carregamento dos Dados
Carregando o dataset no diretório `../data/raw/`.

In [34]:
df_comics.to_csv('../data/raw/comics_corpus.csv', index=False)